In [65]:
# get this data from https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information/data

In [ ]:
# uncomment to reload the data
import pandas as pd
df = pd.read_csv('song_lyrics.csv')
df = df[df['language'] == 'en']
df = df.sort_values('year', ascending=False).head(100000)
df.head()

In [2]:
import time

In [3]:
df.isna().sum()

NameError: name 'df' is not defined

In [8]:
df['language'].value_counts()

language
en    3374198
es     275432
fr     189436
pt     167947
ru     166044
       ...   
mt          5
uz          4
tg          3
bs          1
gu          1
Name: count, Length: 84, dtype: int64

In [ ]:
start = time.time()

df = df.drop(columns=['id', 'language_cld3', 'language_ft'])
df = df[df['language'] == 'en']
df = df.sample(500000, random_state=42)
df.reset_index(drop=True, inplace=True)

end = time.time()
print('Time elapsed:', end - start)

Time elapsed: 247.19573402404785


In [12]:
df['language'].value_counts()

language
en    3374198
Name: count, dtype: int64

In [13]:
df['lyrics']

0          [Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...
1          [Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...
2          Maybe cause I'm eatin\nAnd these bastards fien...
3          [Produced by Kanye West and Brian Miller]\n\n[...
4          [Intro]\nSo they ask me\n"Young boy\nWhat you ...
                                 ...                        
3374193    Everything is alright now\nOh yes, baby\nEvery...
3374194    [Verse 1]\nHalf truth and half you\nDidn't we ...
3374195    [Verse 1]\nDance for me now\nKeeping yourself ...
3374196    [Verse 1]\nHere our purpose feels alive\nWe ar...
3374197    [Verse 1]\nYou need a new number, one that ain...
Name: lyrics, Length: 3374198, dtype: object

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /Users/adviti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/adviti/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adviti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/adviti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/adviti/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def preproc(lyrics):
    lyrics = lyrics.lower()
    lyrics = ''.join(char if char.isalpha() or char.isspace() else ' ' for char in lyrics)
    tokens = word_tokenize(lyrics)
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(cleaned_tokens)

# filter on the mean length of the lyrics
# take only unique words

In [16]:
start = time.time()

df['cleaned_lyrics'] = df['lyrics'].apply(preproc)

end = time.time()
print('Time elapsed:', end - start)

Time elapsed: 3663.372986793518


In [7]:
df.head()

NameError: name 'df' is not defined

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(df['cleaned_lyrics'])

In [ ]:
# use dill.settings['recurse'] = True to save preprocessor

In [5]:
tfidf.shape

NameError: name 'tfidf' is not defined

In [ ]:
# # too heavy to compute cosine similarity for all pairs
# cos_sim = cosine_similarity(tfidf)
# cos_sim

In [3]:
from sklearn.decomposition import TruncatedSVD
import time

In [4]:
n_components = 300
svd = TruncatedSVD(n_components=n_components, random_state=42)

start = time.time()

reduced = svd.fit_transform(tfidf)

end = time.time()

NameError: name 'tfidf' is not defined

In [ ]:
reduced.shape

In [ ]:
cos_sim_reduced = cosine_similarity(reduced)

In [ ]:
df[df['year'] == 2022].sort_values('views', ascending=False)

,title,tag,artist,year,views,features,lyrics,language,cleaned_lyrics
832,Mr. Forgettable,pop,David Kushner,2022,54101,{},[Verse 1]\nI know that you're waiting for me l...,en,verse know waiting like dog patience part lost...
148,Twice,pop,Charli XCX,2022,14502,{},"[Intro]\nDon't think twice, don't think twice\...",en,intro think twice think twice think twice thin...
751,Just a Word,pop,Prateek Kuhad,2022,9924,{},[Verse 1]\nAm I the one that you want?\nMaybe ...,en,verse one want maybe time let know picking flo...
3475,Cupid Remix,pop,2RARE,2022,9352,{},"[Intro]\nOhh, uhh, ayee\nOhh, come on (Uhh, uh...",en,intro ohh uhh ayee ohh come uhh uhh ayee ayee ...
2148,Push Up,rap,Trippie Redd,2022,4805,{Ne-Yo},"[Chorus: Trippie Redd]\nBaby girl, push up on ...",en,chorus trippie redd baby girl push cause want ...
...,...,...,...,...,...,...,...,...,...
2906,​new trend,rap,lilbubblegum,2022,1,{​lilbubblegum},"Lyrics from Snippet\n\nShawty like, ""Who this?...",en,lyric snippet shawty like told new trend kicki...
3315,Im Vibin,country,The Reverent Marigold,2022,1,{},I’m not your potion\nI’m all emotion\nCan’t ha...,en,potion emotion harsh mellow orange yellow blos...
494,​skippity™,rap,mr. master,2022,1,"{""​mr. master""}",[prod. by mr. master]\n\n[intro: saatvik thati...,en,prod mr master intro saatvik thatipally mr mas...
1227,WannaSay,pop,JeiJee,2022,1,{},[Verse]\nI sleep in late\nI'm feeling sick\nHa...,en,verse sleep late feeling sick hard feel good t...


In [ ]:
seed_index = 832

top_match = cos_sim_reduced[seed_index].argsort()[::-1][1]
top_indices = cos_sim_reduced[seed_index].argsort()[::-1][1:6]

In [ ]:
print('Seed Song:')
print(df.loc[seed_index, ['title', 'artist', 'year']])
print(df.loc[seed_index, 'lyrics'][:500])

Seed Song:
title     Mr. Forgettable
artist      David Kushner
year                 2022
Name: 832, dtype: object
[Verse 1]
I know that you're waiting for me like a dog
But have some patience for the part of me that's lost
There's been a hundred times
When I don't recognize
Any of you that love me
I try to memorize and identify
But it's all getting foggy

[Pre-Chorus]
My head is in the clouds right now
Just pray I come around, around

[Chorus]
Hello, hello, are you lonely?
I'm sorry, it's just the chemicals
Hello, hello, do you know me?
I'm called Mr. Forgettable
(Oh, oh, oh, oh-oh, oh-oh, oh-oh)
Mr. Forget


In [ ]:
print('Top Match:')
print(df.loc[top_match, ['title', 'artist', 'year']])
print(df.loc[top_match, 'lyrics'][:500])

Top Match:
title     The Right Stuff The New Kids In the House Mix
artist                            New Kids on the Block
year                                               1990
Name: 3845, dtype: object
[Intro]

[Verse 1]
First time was a great time
Second time was a blast
Third time I fell in love
Now I hope it lasts
I can see it in your walk
Tell it when you talk
See it in everything you do
Even in your thoughts
You got the right stuff, baby
Love the way you turn me on
You got the right stuff, baby
You’re the reason why I sing this song

[Pre-Chorus]
All that I needed was you
Oh girl, you’re so right
Said all that I wanted was you
You made all my dreams come true
[Chorus]
Oh, oh, oh, oh, oh
O


In [ ]:
for index in top_indices:
    print(f"\033[92mSimilar Song {index}:\033[0m")
    print(df.loc[index, ['title', 'artist', 'year']])
    print(df.loc[index, 'lyrics'][:500])

Similar Song 3265:
title         I Believe
artist    David Dreshaj
year               2020
Name: 3265, dtype: object
[Verse 1]
With every touch you make me want you more
And when you're here, I feel like we belong
There is no fear, to you, I give it all
Baby, you're mine, baby, I'm yours

[Pre-Chorus]
With every little touch
You're takin' over me
With everything you do
You make me wanna be
Stronger than I was
And better than I am
I believe in love

[Chorus 1]
I believe in love
I believe, I...
I believe, I believe
Oh, I believe in love
[Verse 2]
With every kiss you make me fall in love
And when you're here, I f
Similar Song 413:
title     ​with you/for you
artist        Prateek Kuhad
year                   2018
Name: 413, dtype: object
Yeah you seem to be okay
Your sweet taste in my mouth stays
So I’ll be waiting in the corner

For you

Not mindless, I’m just spineless
Put Cellophane on my mouth and kiss
My love is lying in the corner

For you, you
It’s true, true

We grow old with thes